# Libraries

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import sklearn as sklearn
from sklearn.preprocessing import StandardScaler

# parameters
input_channels = 5 # open, high, low, close, volume
activation_function = nn.LeakyReLU()
learning_rate = 0.001
epochs = 1000
loss_function = nn.MSELoss()
window_size = 15 # 15 day window
scaler = StandardScaler()

# loading in the dataset

In [18]:
df = pd.read_csv("../Exlai/data.csv")
df = df[df["Company"] == "AAPL"] # select Apple's stock price data

open_data = list()
close_data = list()
high_data = list()
low_data = list()
volume_data = list()

# the dataset is in a shitty format, so i have to fix it here

for i in range(len(df)):
  open_data.append(float(df["Open"][i][1:]))
  close_data.append(float(df["Close/Last"][i][1:]))
  high_data.append(float(df["High"][i][1:]))
  low_data.append(float(df.Low[i][1:]))
  volume_data.append(float(df.Volume[i]))

open_data.reverse()
high_data.reverse()
low_data.reverse()
close_data.reverse()
volume_data.reverse()

In [24]:
data = []

# first we fit our scaler to our mean and standard deviation
scaled_data = scaler.fit_transform(
  torch.tensor([open_data, high_data, low_data, close_data, volume_data]).T
)

# this gets the closing price to predict but we don't want the first windows input data 
target = torch.tensor(scaled_data[window_size:, 3:4], dtype=torch.float)

# this selects our windows and adds them to one list
for i in range(len(open_data) - window_size):
  data.append([
    open_data[i:i + window_size],
    high_data[i:i + window_size], 
    low_data[i:i + window_size], 
    close_data[i:i + window_size], 
    volume_data[i:i + window_size]])

# in order to scale our data we have to swap the rows and columns around
data = torch.tensor(data).permute([0, 2, 1])

# this scales each window to our mean and standard deviation
for i in range(len(data)):
  data[i] = torch.tensor(scaler.transform(data[i]))

# change it back
data = data.permute([0, 2, 1])
  
train_x, test_x, train_y, test_y = sklearn.model_selection.train_test_split(data, target, random_state=7)


# Building the model
The network is set up to output a single value prediction. Probably the closing price for some future date.

In [8]:
class Net(nn.Module): # the nn.Module is set up as the parent class for the Net Class
    def __init__(self):
        super(Net, self).__init__() # This calls the init method of the nn.Module parent class to ensure that its been initialized

        self.model = nn.Sequential(
            nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=2, stride=1),
            activation_function,
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=2, stride=1),
            activation_function,
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(in_features=384, out_features=64),
            activation_function,
            nn.Linear(64, 1),
        )

    def forward(self, x):

        return self.model(x)


net = Net()

Resources: https://www.geeksforgeeks.org/deep-learning/building-a-convolutional-neural-network-using-pytorch/


# Training the model

In [9]:
def train(model: nn.Module, train_x, train_y, epochs, learning_rate):
  optimizer = optim.Adam(net.parameters(), lr=learning_rate)

  for epoch in range(epochs):
    y_hat = net(train_x)
    loss = loss_function(y_hat, train_y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0: # print out the loss as the model is training
      print(f"Epoch: {epoch}, Loss: {loss.item():.10f}")

train(net, train_x, train_y, epochs, learning_rate)


Epoch: 0, Loss: 0.9491673708
Epoch: 50, Loss: 0.0066777593
Epoch: 100, Loss: 0.0043967422
Epoch: 150, Loss: 0.0038152293
Epoch: 200, Loss: 0.0030099468
Epoch: 250, Loss: 0.0023057540
Epoch: 300, Loss: 0.0020730686
Epoch: 350, Loss: 0.0019270654
Epoch: 400, Loss: 0.0018110471
Epoch: 450, Loss: 0.0017144692
Epoch: 500, Loss: 0.0016346531
Epoch: 550, Loss: 0.0015703143
Epoch: 600, Loss: 0.0015207264
Epoch: 650, Loss: 0.0014696572
Epoch: 700, Loss: 0.0014222186
Epoch: 750, Loss: 0.0013797773
Epoch: 800, Loss: 0.0013445887
Epoch: 850, Loss: 0.0013127831
Epoch: 900, Loss: 0.0012802236
Epoch: 950, Loss: 0.0012476660


# Running the model

In [ ]:
y_hat = net(test_x)
loss_function(y_hat, test_y)

price_prediction = y_hat.tolist()
